In [1]:
import qutip, pickle, sys
import matplotlib.pyplot as plt 
import numpy as np
import scipy.optimize as opt 
import scipy.linalg as linalg
import time as time
import math, cmath
import Max_Ent_aux_lib as me
#import proj_ev_library as projev
#import max_entev library as meev
from IPython.display import display, Math, Latex

np.set_printoptions(threshold=1.e-3,linewidth=120,precision=1, formatter={"float":lambda x: str(.001*int(1000*x)) })

Tenemos la siguiente ecuación de movimiento:

$$
    \dot{c}_M(t) I_{M}^{0}(t) = -i \bigg\{\bigg(c_{M+2}(t) - c_M(t) \bigg)\langle \rho_{M}^{0\dagger}(t) [H_{0,-2}, \rho_{M+2}^0] \rangle + \bigg(c_{M-2}(t) - c_M(t)\bigg) \langle \rho_{M}^{0\dagger}(t) [H_{0,2}, \rho_{M-2}^0] \rangle + p \langle \rho_{M}^{0\dagger}(t) [\Sigma, \rho_{M}^0 (t)]\rangle \bigg\}
$$

siendo $I_{M}^{0}(t) = \langle \rho_{M}^{0\dagger}(t) \rho_{M}^{0}(t)\rangle = I_{-M}^{0}(t)$
la cual puede reescribirse como 

$$
\dot{c}_M(t) I_{M}^{0}(t) = -i \bigg\{A_{\rho^{M, M+2}} c_{M+2}(t) - \bigg(A_{\rho^{M, M+2}} + B_{\rho^{M, M-2}} \bigg) c_{M}(t) + B_{\rho^{M, M-2}} c_{M-2}(t) \bigg\} + p C_{\rho^M} \\
= - i({\bf{\mathcal{M}}}(t) \textbf{c}(t))_M
$$

donde 

<ol>

<li> $A_{\rho^{M, M+2}} = \langle \rho_{M}^{0\dagger}(t) [H_{0,-2}, \rho_{M+2}^0] \rangle$ </li> 
<li> $B_{\rho^{M, M-2}} = \langle \rho_{M}^{0\dagger}(t) [H_{0,2}, \rho_{M-2}^0] \rangle$ </li>
<li> $C_{\rho^M} = \langle \rho_{M}^{0\dagger}(t) [\Sigma, \rho_{M}^0 (t)]\rangle$ </li>
    
</ol>    

Entonces, si $f(t) = \langle \rho(t) \rho^0(t) \rangle = \sum_{M} \dot{c}_M(t) I_{M}^{0}(t)$

$$
    \dot{f}(t) = \sum_{M} \bigg(\dot{c}_M(t) I_{M}^{0}(t) + {c}_M(t) \dot{I}_{M}^{0}(t) \bigg)
$$

where

1: $A=\alpha B = N(t) \exp(-m t^{1+a}) = I_M^{(0)}$ (a primer orden alpha = 1) \
2: $C = M N(t) exp(-m t^{1+a})$

con $N(t) = 1/Tr(\rho(t))$ la calculo con la definición del $\rho_M (t)=  c_{M}(t) \rho^0_M(t)$, with $c_0(t)$= 1. 
Note that the kernel $K(t,t') = e^{(-i M(t-t'))}$ is not a solution to the previous differential equation for the M-tensor is time-dependent
    
$H_ {0, 2}^\dagger = H_{0, -2}$ 

$\rho_{M}^\dagger = \rho_{-M}$

## Step 1: Fix parameters and initial conditions for the coherences

In [4]:
p = .0009;    # strength of the Sigma Interaction Hamiltonian
a = -.05;      # Power-law factor
M = 625;        # Truncation/Total no. of coherences
coherences_t0_pert0 = 1. # Unused for the time being
param_list = {"total_no_cohrs": M, "p_factor": p, "power_law_factor": a} # dictionary containing the simulation's initial 
                                                                         # parameters 

coeff_list_t0 = np.array([coherences_t0_pert0 - np.random.rand() for i in range(param_list["total_no_cohrs"])])
              # initial configuration for the coherences, random numbers for the time being. Can be changed in the future

coeff_list_t0 = coeff_list_t0/(sum(coeff_list_t0)) 
              # normalization of the initial vector, so that Tr c_0 = 1. 

Since the coherences are complex-valued numbers, the previous system of $M \times M$ coupled complex-valued differential equations can be rewritten as a system of $2M \times 2M$ coupled real-valued differential equations, as follows:

if $c_M(t) = a_M(t) + i b_M(t)$, then 

$$
    \dot{a}_M(t) + i \dot{b}_M(t) = -i \sum_{m'}{\bf{\mathcal{M}}}_{Mm'}(t) \bigg(a_{m'}(t) + i b_{m'}(t)\bigg),
$$

$$
    \dot{a}_M(t) + i \dot{b}_M(t) = -i \sum_{m'}{\bf{\mathcal{M}}}_{Mm'}(t) a_{m'}(t) + \sum_{m'} {\bf{\mathcal{M}}}_{Mm'}(t) b_{m'}(t)
$$

$$
    \dot{{\bf a}}(t) = {\cal M}(t) \cdot {\bf b}(t), \quad \dot{{\bf b}}(t) = -{\cal M}(t) \cdot {\bf a}(t), \qquad s.t. \quad{\bf c}(t) = {\bf a}(t) + i {\bf b}(t)
$$


## Step 2: Setting up system of complex diff. eqs

In [9]:
def Mtensor_2mx2m_dimensional_symplectic(parameters, init_configurations, timet, 
                                            closed_boundary_conditions = False,
                                            visualization = False,
                                            as_qutip_qobj = False):
    """
     
    """
    M = parameters["total_no_cohrs"]; p = parameters["p_factor"]; a = parameters["power_law_factor"]
    m_matrix_list = []; t = timet;
    
    m_dimensional_zero_array = [0 for m in range(M)]
    
    for m in range(M):
        if m == 0:
            local_array = np.array(m_dimensional_zero_array + [me.A_mmplustwo_matrix_elmt(cohrnc = m, time = t, 
                                                                   power_law_factor = a) + 
                                           p * me.C_m_matrix_elmt(cohrnc = m, time = t, 
                                                                   power_law_factor = a)] 
                                          + [me.A_mmplustwo_matrix_elmt(cohrnc = m+2, time = t, 
                                                                   power_law_factor = a)] 
                                          + [0 for i in range(M-2)])
            local_array = local_array/(sum(local_array))
            m_matrix_list.append(local_array)
            local_array = None
        if (m > 0 and m < M-1):
            list_with_zeros = [0 for i in range(m-1)]
            local_array = np.array(m_dimensional_zero_array + list_with_zeros
                                          + [me.B_mmminustwo_matrix_elmt(cohrnc = m, time = t, power_law_factor = a)]
                                          + [me.diag_mm_matrix_elmt(cohrnc = m, time = t, power_law_factor = a, p = p)] 
                                          + [me.A_mmplustwo_matrix_elmt(cohrnc = m+2, time = t, power_law_factor = a)]  
                                          + [0 for i in range(M - (len(list_with_zeros)+3))])
            local_array = local_array/(sum(local_array))
            m_matrix_list.append(local_array)
            local_array = None       
        if (m == M-1):
            local_array = np.array(m_dimensional_zero_array + [0 for i in range(M - 2)] 
                                          + [me.B_mmminustwo_matrix_elmt(cohrnc = m, time = t, power_law_factor = a)] 
                                          + [me.B_mmminustwo_matrix_elmt(cohrnc = m, time = t, power_law_factor = a) + 
                                               p * me.C_m_matrix_elmt(cohrnc = m, time = t, power_law_factor = a)])
            local_array = local_array/(sum(local_array))
            m_matrix_list.append(local_array)
            local_array = None
            
    for m in range(M):
        if m == 0:
            local_array = np.array([me.A_mmplustwo_matrix_elmt(cohrnc = m, time = t, 
                                                                   power_law_factor = a) + 
                                           p * me.C_m_matrix_elmt(cohrnc = m, time = t, 
                                                                   power_law_factor = a)] 
                                          + [me.A_mmplustwo_matrix_elmt(cohrnc = m+2, time = t, 
                                                                   power_law_factor = a)] 
                                          + [0 for i in range(M-2)]
                                + m_dimensional_zero_array)
            local_array = local_array/(sum(local_array))
            m_matrix_list.append(local_array)
            local_array = None
        if (m > 0 and m < M-1):
            list_with_zeros = [0 for i in range(m-1)]
            local_array = np.array(list_with_zeros
                                          + [me.B_mmminustwo_matrix_elmt(cohrnc = m, time = t, power_law_factor = a)]
                                          + [me.diag_mm_matrix_elmt(cohrnc = m, time = t, power_law_factor = a, p = p)] 
                                          + [me.A_mmplustwo_matrix_elmt(cohrnc = m+2, time = t, power_law_factor = a)]  
                                          + [0 for i in range(M - (len(list_with_zeros)+3))]
                                + m_dimensional_zero_array)
            local_array = local_array/(sum(local_array))
            m_matrix_list.append(local_array)
            local_array = None       
        if (m == M-1):
            local_array = np.array([0 for i in range(M - 2)] 
                                          + [me.B_mmminustwo_matrix_elmt(cohrnc = m, time = t, power_law_factor = a)] 
                                          + [me.B_mmminustwo_matrix_elmt(cohrnc = m, time = t, power_law_factor = a) + 
                                               p * me.C_m_matrix_elmt(cohrnc = m, time = t, power_law_factor = a)]
                                + m_dimensional_zero_array)
            local_array = local_array/(sum(local_array))
            m_matrix_list.append(local_array)
            local_array = None
    
    return m_matrix_list

## Step 3: Solving the system

In [10]:
ts = np.linspace(0.1, 1., 100)
y0 = coeff_list_t0

def complex_differential_system(cohr_complex, t, parameters):
    Mtensor = Mtensor_2mx2m_dimensional_symplectic(parameters = parameters, init_configurations = cohr_complex, timet = t, 
                                                closed_boundary_conditions = False,
                                                visualization = False,
                                                as_qutip_qobj = False)
    return Mtensor @ cohr_complex

from scipy.integrate import odeint

cohr_complex_t0 = [coherences_t0_pert0 - np.random.rand() for i in range(param_list["total_no_cohrs"])]
cohr_complex_t0 += [0 for i in range(param_list["total_no_cohrs"])]
cohr_complex_t0 = np.array(cohr_complex_t0)


In [11]:
B = complex_differential_system(cohr_complex = cohr_complex_t0, t=1, parameters = param_list)
len(B)

1250

In [89]:
ts = np.linspace(0.1, 1., 100)              ## times 
result = odeint(func = complex_differential_system, 
                y0 = cohr_complex_t0, 
                t = ts,
                args = ((param_list,)))
result

array([[0.883, 0.664, 0.137, ..., 0.0, 0.0, 0.0],
       [0.883, 0.664, 0.137, ..., 0.005, 0.005, 0.005],
       [0.883, 0.664, 0.137, ..., 0.01, 0.01, 0.01],
       ...,
       [0.9450000000000001, 1.194, 0.253, ..., 0.5710000000000001, 0.591, 0.554],
       [0.9460000000000001, 1.206, 0.257, ..., 0.578, 0.598, 0.561],
       [0.9480000000000001, 1.217, 0.26, ..., 0.586, 0.606, 0.5690000000000001]])

In [119]:
result[:, 1][int(list(ts).index(0.1))]

0.6642325096497997

In [127]:
fidelity_vs_t = [[sum(result[:, m][int(list(ts).index(t)) 
                                   * -m * (a+1) * t**a / (me.A_mmplustwo_matrix_elmt(m, t, a))] for m in range(M))] for t in list(ts)]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [72]:
len(result[:, 0])

100

In [73]:
len(result[0])

6

# Tests y fallos :(

In [23]:
Preguntas:
    
    1. La matrix M tiene que depender con el tiempo me parece, si A, B y C lo hacen: Check 
    2. Como tendría que implementar la norma???: Check
    3. Empezar a jugar con los parámetros: Checkn't
    
m+2 vs m+2 : Check
** reescribir la matriz en tèrminos de los c_pares. : Check

SyntaxError: invalid syntax (3991330021.py, line 1)